In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
2212885,2022-03-13,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
2212886,2022-03-14,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
2212887,2022-03-15,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
2212888,2022-03-16,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
56959,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
56961,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
56963,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
56965,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
56967,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
1340942,2022-03-13,Arkansas,Arkansas,5001,5662,78.00,5000,Arkansas,AR,Arkansas,State,3017804
1340944,2022-03-14,Arkansas,Arkansas,5001,5663,78.00,5000,Arkansas,AR,Arkansas,State,3017804
1340946,2022-03-15,Arkansas,Arkansas,5001,5663,78.00,5000,Arkansas,AR,Arkansas,State,3017804
1340948,2022-03-16,Arkansas,Arkansas,5001,5666,78.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
2212885,2022-03-13,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
2212886,2022-03-14,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
2212887,2022-03-15,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
2212888,2022-03-16,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2022-03-17') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
786,2022-03-17,Snohomish,Washington,148174,1093.00,53061,WA,County,822083,18024.21,132.95
1570,2022-03-17,Cook,Illinois,1119976,14750.00,17031,IL,County,5150233,21746.12,286.39
2353,2022-03-17,Orange,California,584517,6820.00,6059,CA,County,3175692,18405.97,214.76
3135,2022-03-17,Maricopa,Arizona,1256653,16166.00,4013,AZ,County,4485414,28016.43,360.41
3917,2022-03-17,Los Angeles,California,2817894,31408.00,6037,CA,County,10039107,28069.17,312.86
...,...,...,...,...,...,...,...,...,...,...,...
2210929,2022-03-17,Wheeler,Oregon,243,3.00,41069,OR,County,1332,18243.24,225.23
2211450,2022-03-17,King,Texas,41,0.00,48269,TX,County,272,15073.53,0.00
2211940,2022-03-17,Esmeralda,Nevada,90,3.00,32009,NV,County,873,10309.28,343.64
2212426,2022-03-17,Loving,Texas,167,1.00,48301,TX,County,169,98816.57,591.72


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
786,2022-03-17,Snohomish,Washington,148174,1093.00,53061,WA,County,822083,18024.21,132.95,132.95,18024.21
1570,2022-03-17,Cook,Illinois,1119976,14750.00,17031,IL,County,5150233,21746.12,286.39,286.39,21746.12
2353,2022-03-17,Orange,California,584517,6820.00,6059,CA,County,3175692,18405.97,214.76,214.76,18405.97
3135,2022-03-17,Maricopa,Arizona,1256653,16166.00,4013,AZ,County,4485414,28016.43,360.41,360.41,28016.43
3917,2022-03-17,Los Angeles,California,2817894,31408.00,6037,CA,County,10039107,28069.17,312.86,312.86,28069.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2210929,2022-03-17,Wheeler,Oregon,243,3.00,41069,OR,County,1332,18243.24,225.23,225.23,18243.24
2211450,2022-03-17,King,Texas,41,0.00,48269,TX,County,272,15073.53,0.00,0.00,15073.53
2211940,2022-03-17,Esmeralda,Nevada,90,3.00,32009,NV,County,873,10309.28,343.64,343.64,10309.28
2212426,2022-03-17,Loving,Texas,167,1.00,48301,TX,County,169,98816.57,591.72,591.72,98816.57


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
1183160,2022-03-17,Winston,Alabama,7569,126.00,1133,AL,County,23629,32032.67,533.24,533.24,32032.67,1
1402781,2022-03-17,Hale,Alabama,4682,103.00,1065,AL,County,14651,31956.86,703.02,703.02,31956.86,2
812826,2022-03-17,Franklin,Alabama,9844,137.00,1059,AL,County,31362,31388.30,436.83,436.83,31388.30,3
967116,2022-03-17,Clay,Alabama,4086,81.00,1027,AL,County,13235,30872.69,612.01,612.01,30872.69,4
1402063,2022-03-17,Fayette,Alabama,4970,93.00,1057,AL,County,16302,30487.06,570.48,570.48,30487.06,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1894140,2022-03-17,Big Horn,Wyoming,2575,60.00,56003,WY,County,11790,21840.54,508.91,508.91,21840.54,19
1179555,2022-03-17,Johnson,Wyoming,1842,19.00,56019,WY,County,8445,21811.72,224.99,224.99,21811.72,20
1667778,2022-03-17,Lincoln,Wyoming,3966,29.00,56023,WY,County,19830,20000.00,146.24,146.24,20000.00,21
1338793,2022-03-17,Sublette,Wyoming,1923,27.00,56035,WY,County,9831,19560.57,274.64,274.64,19560.57,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
1095081,2022-03-17,Lowndes,Alabama,2606,75.00,1085,AL,County,9726,26794.16,771.13,771.13,26794.16,31,1
1402781,2022-03-17,Hale,Alabama,4682,103.00,1065,AL,County,14651,31956.86,703.02,703.02,31956.86,2,2
1093637,2022-03-17,Crenshaw,Alabama,3784,96.00,1041,AL,County,13772,27476.04,697.07,697.07,27476.04,25,3
462859,2022-03-17,Walker,Alabama,18524,429.00,1127,AL,County,63521,29162.01,675.37,675.37,29162.01,10,4
964947,2022-03-17,Butler,Alabama,5045,125.00,1013,AL,County,19448,25940.97,642.74,642.74,25940.97,35,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1179555,2022-03-17,Johnson,Wyoming,1842,19.00,56019,WY,County,8445,21811.72,224.99,224.99,21811.72,20,19
1625652,2022-03-17,Uinta,Wyoming,5637,39.00,56041,WY,County,20226,27870.07,192.82,192.82,27870.07,5,20
1667778,2022-03-17,Lincoln,Wyoming,3966,29.00,56023,WY,County,19830,20000.00,146.24,146.24,20000.00,21,21
1090749,2022-03-17,Albany,Wyoming,9847,48.00,56001,WY,County,38880,25326.65,123.46,123.46,25326.65,14,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,132.95,18024.21,31,20
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,132.95,18024.21,31,20
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,132.95,18024.21,31,20
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,132.95,18024.21,31,20
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,132.95,18024.21,31,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2207782,2022-03-13,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
2207783,2022-03-14,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
2207784,2022-03-15,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
2207785,2022-03-16,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa[ '14day_case_avg' ] = usa.case_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg
1178777,2020-03-27,Winston,Alabama,2,0.00,1133,AL,County,23629,8.46,0.00,533.24,32032.67,17,1,2.00,nan
1178778,2020-03-28,Winston,Alabama,2,0.00,1133,AL,County,23629,8.46,0.00,533.24,32032.67,17,1,0.00,nan
1178779,2020-03-29,Winston,Alabama,2,0.00,1133,AL,County,23629,8.46,0.00,533.24,32032.67,17,1,0.00,nan
1178780,2020-03-30,Winston,Alabama,2,0.00,1133,AL,County,23629,8.46,0.00,533.24,32032.67,17,1,0.00,nan
1178781,2020-03-31,Winston,Alabama,2,0.00,1133,AL,County,23629,8.46,0.00,533.24,32032.67,17,1,0.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1696182,2022-03-13,Crook,Wyoming,1346,28.00,56011,WY,County,7584,17747.89,369.20,369.20,17747.89,9,23,0.00,0.71
1696183,2022-03-14,Crook,Wyoming,1346,28.00,56011,WY,County,7584,17747.89,369.20,369.20,17747.89,9,23,0.00,0.50
1696184,2022-03-15,Crook,Wyoming,1346,28.00,56011,WY,County,7584,17747.89,369.20,369.20,17747.89,9,23,0.00,0.43
1696185,2022-03-16,Crook,Wyoming,1346,28.00,56011,WY,County,7584,17747.89,369.20,369.20,17747.89,9,23,0.00,0.36


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa[ '14day_death_avg' ] = usa.death_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
1090697,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,771.13,26794.16,1,31,1.00,3.07,0.00,nan
1090698,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,771.13,26794.16,1,31,0.00,3.07,0.00,nan
1090699,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,771.13,26794.16,1,31,0.00,3.07,0.00,nan
1090700,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,771.13,26794.16,1,31,0.00,2.00,0.00,nan
1090701,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,771.13,26794.16,1,31,0.00,1.79,0.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457728,2022-03-13,Teton,Wyoming,9842,16.00,56039,WY,County,23464,41945.11,68.19,68.19,41983.46,23,1,0.00,5.36,0.00,0.00
457729,2022-03-14,Teton,Wyoming,9842,16.00,56039,WY,County,23464,41945.11,68.19,68.19,41983.46,23,1,0.00,4.29,0.00,0.00
457730,2022-03-15,Teton,Wyoming,9851,16.00,56039,WY,County,23464,41983.46,68.19,68.19,41983.46,23,1,9.00,4.29,0.00,0.00
457731,2022-03-16,Teton,Wyoming,9851,16.00,56039,WY,County,23464,41983.46,68.19,68.19,41983.46,23,1,0.00,4.07,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2022-03-17') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
553223,2022-03-17,Imperial,California,66005,896.00,6025,CA,County,181215,36423.59,494.44,494.44,36423.59,1,1,99.00,31.36,1.00,1.36
1339442,2022-03-17,Tuolumne,California,13349,180.00,6109,CA,County,54478,24503.47,330.41,330.41,24503.47,2,14,0.00,27.57,0.00,0.07
84353,2022-03-17,Shasta,California,36650,587.00,6089,CA,County,180080,20352.07,325.97,325.97,20352.07,3,28,3.00,19.43,0.00,2.00
3917,2022-03-17,Los Angeles,California,2817894,31408.00,6037,CA,County,10039107,28069.17,312.86,312.86,28069.17,4,5,1811.00,1174.93,36.00,35.50
274203,2022-03-17,San Bernardino,California,584060,6775.00,6071,CA,County,2180085,26790.70,310.77,310.77,26790.70,5,8,290.00,214.14,5.00,11.14
972852,2022-03-17,Inyo,California,4543,56.00,6027,CA,County,18039,25184.32,310.44,310.44,25184.32,6,10,0.00,2.00,0.00,0.00
122005,2022-03-17,Stanislaus,California,136645,1694.00,6099,CA,County,550660,24814.77,307.63,307.63,24814.77,7,13,0.00,60.07,0.00,1.93
151482,2022-03-17,Tulare,California,132387,1382.00,6107,CA,County,466195,28397.34,296.44,296.44,28397.34,8,4,102.00,66.36,7.00,5.71
880921,2022-03-17,Merced,California,71501,805.00,6047,CA,County,277680,25749.42,289.90,289.90,25749.42,9,9,52.00,38.43,2.00,0.79
1181660,2022-03-17,Kings,California,54936,440.00,6031,CA,County,152940,35919.97,287.69,287.69,35919.97,10,2,13.00,38.50,3.00,0.64


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
553223,2022-03-17,Imperial,California,66005,896.00,6025,CA,County,181215,36423.59,494.44,494.44,36423.59,1,1,99.00,31.36,1.00,1.36
1181660,2022-03-17,Kings,California,54936,440.00,6031,CA,County,152940,35919.97,287.69,287.69,35919.97,10,2,13.00,38.50,3.00,0.64
2082004,2022-03-17,Lassen,California,10695,68.00,6035,CA,County,30573,34981.85,222.42,222.42,34981.85,21,3,4.00,2.07,0.00,0.07
151482,2022-03-17,Tulare,California,132387,1382.00,6107,CA,County,466195,28397.34,296.44,296.44,28397.34,8,4,102.00,66.36,7.00,5.71
3917,2022-03-17,Los Angeles,California,2817894,31408.00,6037,CA,County,10039107,28069.17,312.86,312.86,28069.17,4,5,1811.00,1174.93,36.00,35.50
55476,2022-03-17,Madera,California,43282,354.00,6039,CA,County,157327,27510.85,225.01,225.01,27510.85,20,6,9.00,16.43,0.00,0.00
345249,2022-03-17,Kern,California,241765,2222.00,6029,CA,County,900202,26856.75,246.83,246.83,26856.75,15,7,108.00,207.07,18.00,6.14
274203,2022-03-17,San Bernardino,California,584060,6775.00,6071,CA,County,2180085,26790.70,310.77,310.77,26790.70,5,8,290.00,214.14,5.00,11.14
880921,2022-03-17,Merced,California,71501,805.00,6047,CA,County,277680,25749.42,289.90,289.90,25749.42,9,9,52.00,38.43,2.00,0.79
972852,2022-03-17,Inyo,California,4543,56.00,6027,CA,County,18039,25184.32,310.44,310.44,25184.32,6,10,0.00,2.00,0.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
553223,2022-03-17,Imperial,California,66005,896.00,6025,CA,County,181215,36423.59,494.44,494.44,36423.59,1,1,99.00,31.36,1.00,1.36
1339442,2022-03-17,Tuolumne,California,13349,180.00,6109,CA,County,54478,24503.47,330.41,330.41,24503.47,2,14,0.00,27.57,0.00,0.07
84353,2022-03-17,Shasta,California,36650,587.00,6089,CA,County,180080,20352.07,325.97,325.97,20352.07,3,28,3.00,19.43,0.00,2.00
3917,2022-03-17,Los Angeles,California,2817894,31408.00,6037,CA,County,10039107,28069.17,312.86,312.86,28069.17,4,5,1811.00,1174.93,36.00,35.50
274203,2022-03-17,San Bernardino,California,584060,6775.00,6071,CA,County,2180085,26790.70,310.77,310.77,26790.70,5,8,290.00,214.14,5.00,11.14
972852,2022-03-17,Inyo,California,4543,56.00,6027,CA,County,18039,25184.32,310.44,310.44,25184.32,6,10,0.00,2.00,0.00,0.00
122005,2022-03-17,Stanislaus,California,136645,1694.00,6099,CA,County,550660,24814.77,307.63,307.63,24814.77,7,13,0.00,60.07,0.00,1.93
151482,2022-03-17,Tulare,California,132387,1382.00,6107,CA,County,466195,28397.34,296.44,296.44,28397.34,8,4,102.00,66.36,7.00,5.71
880921,2022-03-17,Merced,California,71501,805.00,6047,CA,County,277680,25749.42,289.90,289.90,25749.42,9,9,52.00,38.43,2.00,0.79
1181660,2022-03-17,Kings,California,54936,440.00,6031,CA,County,152940,35919.97,287.69,287.69,35919.97,10,2,13.00,38.50,3.00,0.64


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'current_crate', 'state_case', 'case_day', '14day_case_avg']]
top10_ca_case_df

,date,county,cases,current_crate,state_case,case_day,14day_case_avg
0,03/20/20,Imperial,4,2.21,1,4.00,1.29
1,03/21/20,Imperial,4,2.21,1,0.00,1.07
2,03/22/20,Imperial,4,2.21,1,0.00,1.07
3,03/23/20,Imperial,4,2.21,1,0.00,1.07
4,03/24/20,Imperial,9,4.97,1,5.00,1.43
...,...,...,...,...,...,...,...
7278,03/13/22,Lassen,10689,34962.22,3,0.00,3.71
7279,03/14/22,Lassen,10689,34962.22,3,0.00,3.71
7280,03/15/22,Lassen,10690,34965.49,3,1.00,3.00
7281,03/16/22,Lassen,10691,34968.76,3,1.00,2.07


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','deaths', 'current_drate', 'state_death', 'death_day', '14day_death_avg']]
top10_ca_death_df

,date,county,deaths,current_drate,state_death,death_day,14day_death_avg
0,03/20/20,Imperial,0.00,0.00,1,0.00,0.00
1,03/21/20,Imperial,0.00,0.00,1,0.00,0.00
2,03/22/20,Imperial,0.00,0.00,1,0.00,0.00
3,03/23/20,Imperial,0.00,0.00,1,0.00,0.00
4,03/24/20,Imperial,0.00,0.00,1,0.00,0.00
...,...,...,...,...,...,...,...
7337,03/13/22,Kings,436.00,285.08,10,0.00,0.71
7338,03/14/22,Kings,437.00,285.73,10,1.00,0.64
7339,03/15/22,Kings,437.00,285.73,10,0.00,0.71
7340,03/16/22,Kings,437.00,285.73,10,0.00,0.50


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)